In [1]:
#!/usr/bin/python
# updated DBR 02/2025 #

%matplotlib inline  

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import scipy.optimize as opt #for power law fitting
import time
import pickle

#ipython magic to make module autoreload
%load_ext autoreload
%autoreload 2

#pull in my modules code to do the sims!
import ra_module
import clonesim_module1 as clmod

#for parallel stuff
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()
print(f"Available CPU cores: {num_cores}")

Available CPU cores: 8


In [2]:
#initialize the base model
#uniform T0 with T0 model param:100 (T0i), uniform proliferation, no reemergence, never redraw rates, no xi, full coverage

#HIV provirus proportions, from IPDA data, per million CD4
T0=1e6
cHIV = 3000*T0/1e6; 
cHIVint = 800*T0/1e6; 
cHIVsurv = 0 #300*T0/1e6 #1/100 intact? total? have survival advantages?

#decay rates
xi_int = 0
xi_def = 0
zeta   = 0.001 #additional advantage, should basically balance out xi

model = { 'fn':'1',       #name for saving
          'dt':30,        #time step, start monthly
          'tF':365*10,    #sim length days
          'T0':int(T0),  #initial TCR size, needs to be an integer 
          'm0':'uni',     #type of initial condition distribution ['const','uni','exp','pwl']
          'Tmp':100,      #parameterize initial size distribution ['const','uni','exp','pwl']
          'rm':'uni',     #model for proliferation events, options: ['uni','exp','pwl','2phase']
          'rmp':0,        #parameterize distribution of proliferation rates disrtribution (a list if 2phase is chosen)
          'apr':0.01,     #avg prolif rate, per day, TCM rate
          'ncr':0,        #net clearance rate, balance for T cells, net = prolif - death
          'aer':0,        #emergence rate cells/day, new
          'trd':20000,    #time to redraw rates, if >tF not happen
          'cov':1,        #start with HIV fully covers clones
          'pps':[cHIV,cHIVint,cHIVsurv],      #provirus proportions
          'pds':[xi_int,xi_def,zeta]}     #provirus decay rates


In [8]:
#all model combinations!!

dfM=pd.DataFrame()

model_l = []; im=0

model_change = []
model_change_num = []

model['m0']='uni'
for iTmp, Tmp in enumerate([10,100]):
    model['Tmp']=Tmp
    for irm,rm in enumerate(['const','uni','exp']):
        model['rm']=rm
        if rm=='exp':
            model['rmp']=1

        for itrd,trd in enumerate([20000,50]):
            model['trd']=trd

            for ixi,xi_int in enumerate([0,np.log(2)/(40*30)]):
                model['pds']=[xi_int,xi_def,zeta]

                for ixd,xi_def in enumerate([0,np.log(2)/(100*30)]):
                    model['pds']=[xi_int,xi_def,zeta]

                    for iHc,Hc in enumerate([1,0]):
                        model['cov']=Hc

                        for iae,aer in enumerate([0,0.1]):
                            model['aer']=aer

                            model['fn']=str(im)
                            model_l.append(model.copy())
                            im+=1

                            model_change.append([model['m0'],Tmp,rm,trd,xi_int,xi_def,aer,Hc])
                            model_change_num.append([0,irm,itrd,ixi,ixd,iae,iHc])

model['m0']='pwl'
for iTmp, Tmp in enumerate(np.arange(0.3,1.1,0.1)):
    model['Tmp']=Tmp
    for irm,rm in enumerate(['const','uni','exp']):
        model['rm']=rm
        if rm=='exp':
            model['rmp']=1

        for itrd,trd in enumerate([20000,50]):
            model['trd']=trd

            for ixi,xi_int in enumerate([0,np.log(2)/(40*30)]):
                model['pds']=[xi_int,xi_def,zeta]

                for ixd,xi_def in enumerate([0,np.log(2)/(100*30)]):
                    model['pds']=[xi_int,xi_def,zeta]

                    for iHc,Hc in enumerate([1,0]):
                        model['cov']=Hc

                        for iae,aer in enumerate([0,0.1]):
                            model['aer']=aer

                            model['fn']=str(im)
                            model_l.append(model.copy())
                            im+=1

                            model_change.append([model['m0'],Tmp,rm,trd,xi_int,xi_def,aer,Hc])
                            model_change_num.append([1,irm,itrd,ixi,ixd,iae,iHc])


In [9]:
len(model_l)


960

In [10]:
#RUN IT
#np.seterr(divide = 'ignore') #to deal with spitting out pink boxes??

models_to_run = model_l

start = time.time()
rez = Parallel(n_jobs=8)(delayed(clmod.run_full)(m) for m in models_to_run)
end = time.time()

print(len(models_to_run),'models took',np.round((end - start)),'s')


960 models took 2102.0 s


In [11]:
#output results
sim_name = 'parallel_full2'

mcdf = pd.DataFrame(model_change[:len(models_to_run)],
                    columns=['m0','Tmp','rm','trd','xi_int','xi_def','Hc','aer'])
mcdf_num = pd.DataFrame(model_change_num[:len(models_to_run)],
                        columns=['m0','rm','trd','xi_int','xi_def','Hc','aer'])

# Save to a pickle file, just keeps in df format
with open('models-'+sim_name+'.pkl', 'wb') as f:
    pickle.dump(model_l, f)

with open('model_change-'+sim_name+'.pkl', 'wb') as f:
    pickle.dump(mcdf, f)

with open('model_change_num-'+sim_name+'.pkl', 'wb') as f:
    pickle.dump(mcdf_num, f)

dfM = pd.DataFrame(rez)
dfM.to_csv('model_scores-'+sim_name+'.csv')
dfM.head()



,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.173728,0.249349,0.410912,1.313793,1.451034,-0.811442,-0.583626,-0.000619,0.001237,-0.523626,...,0.730140,0.6663,0.582756,0.496672,0.6663,0.582756,0.496672,0.6663,0.582756,0.496672
1,0.165511,0.251523,0.405154,1.313793,1.451034,-0.793121,-0.599860,-0.003196,-0.001156,-0.548371,...,0.728332,0.6707,0.588527,0.501756,0.6707,0.588527,0.501756,0.6707,0.588527,0.501756
2,0.172657,0.246688,0.407149,1.245172,1.313793,-0.611163,-0.434704,0.000918,0.000149,-0.537204,...,0.730674,0.6679,0.586457,0.503170,0.6679,0.586457,0.503170,0.6679,0.586457,0.503170
3,0.174223,0.242704,0.411290,1.245172,1.313793,-0.601952,-0.340095,0.003416,0.000818,-0.550798,...,0.736811,0.6607,0.578675,0.495540,0.6607,0.578675,0.495540,0.6607,0.578675,0.495540
4,0.175377,0.251363,0.407385,1.313793,1.382414,-0.772786,-0.383110,0.002355,-0.002157,-0.534466,...,0.726322,0.6671,0.585540,0.501958,0.6671,0.585540,0.501958,0.6671,0.585540,0.501958


In [ ]:
#672 models took 1402.0 s!!